# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.128549e+02     1.547696e+00
 * time: 0.6552150249481201
     1     1.046703e+01     8.079784e-01
 * time: 1.8050251007080078
     2    -1.226056e+01     9.431401e-01
 * time: 2.374380111694336
     3    -3.436714e+01     7.800122e-01
 * time: 3.140610933303833
     4    -4.779856e+01     5.783946e-01
 * time: 3.917232036590576
     5    -5.703136e+01     1.668315e-01
 * time: 4.698923110961914
     6    -5.982137e+01     1.103775e-01
 * time: 5.216047048568726
     7    -6.083635e+01     5.659195e-02
 * time: 5.722250938415527
     8    -6.122631e+01     7.377035e-02
 * time: 6.227930068969727
     9    -6.153438e+01     3.269252e-02
 * time: 6.787600994110107
    10    -6.174967e+01     2.466485e-02
 * time: 7.313355922698975
    11    -6.191940e+01     1.854978e-02
 * time: 7.835582971572876
    12    -6.199006e+01     1.573548e-02
 * time: 8.36442494392395
    13    -6.206501e+01     1.515734e-02
 * time: 8.878807067871094
    14

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671078
    AtomicLocal         -18.8557726
    AtomicNonlocal      14.8522675
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485385 
    Xc                  -19.3336825

    total               -62.261666458483